## Final Project Submission

Please fill out:
* Student name: Max Steele
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


# Introduction

Through this analysis, I will seek to answer the following questions:
1. Which genres are top grossing worldwide and have the highest return on investment?
* Of the top grossing genres, how does movie budget affect return on investment?
* Of the top grossing genres, how does MPAA rating influence return on investment?

# Create DataFrames from IMDb csv and tsv Files

In [369]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gzip
import csv
import json
import requests
import sqlite3
import csv_to_sqlite
from pandasql import sqldf

This function is helpful for quickly and easily inspecting and summarizing some key features of the data. It returns a DataFrame reporting the datatype of each column, the number of null values in each column as well as the percent of the column that is null values, as well as the number of unique values found in each column.

In [370]:
def check_df(df):
    info = {'dtypes':df.dtypes,
           'null values':df.isna().sum(),
            '% null': np.round((df.isna().sum()/len(df))*100,2),
           'nunique':df.nunique(),}
    return pd.DataFrame(info).head(len(df.columns))

In [371]:
def check_ends(df):
    return (display(df.shape, df.head(), df.tail()))

In [372]:
## find the files in my directory
import os
os.listdir('imdbZippedData/')

['title.principals.tsv.gz',
 'title.akas.tsv.gz',
 'title.basics.tsv.gz',
 'title.crew.tsv.gz',
 'title.ratings.tsv.gz',
 'name.basics.tsv.gz']

In [373]:
## import files as a list
import glob
file_list = glob.glob('imdbZippedData/*sv*')
file_list

['imdbZippedData/title.principals.tsv.gz',
 'imdbZippedData/title.akas.tsv.gz',
 'imdbZippedData/title.basics.tsv.gz',
 'imdbZippedData/title.crew.tsv.gz',
 'imdbZippedData/title.ratings.tsv.gz',
 'imdbZippedData/name.basics.tsv.gz']

In [374]:
## test how to adjust file name to serve as an informative key name
file_list[0].split('/')[-1].replace('.tsv.gz', '')

'title.principals'

In [375]:
## create an empty dictionary data tables from files
TABLES = {}

## loop through my list of files
for file in file_list:
    
    ## if file name ends with .tsv.gz, read and separate by tabs
    try:
        if file.endswith('tsv.gz'):
            df = pd.read_csv(file, sep='\t', encoding = "latin-1", low_memory=False)
            key = file.split('/')[-1].replace('.tsv.gz', '').replace('.',"_")
     
    ## otherwise read the file as comma separated with defaults   
        else:
            df = pd.read_csv(file, low_memory=False)
            key = file.split('/')[-1].replace('.csv.gz', '').replace('.',"_")
        
    ## if the above raises an error (due to issue with UTF-8 encoding), change from default encoding to latin-1
    ## and read by separating by tabs and set key name based on file name
    except UnicodeDecodeError:
        df = pd.read_csv(file, sep='\t', encoding = "latin-1", low_memory=False)
        key = file.split('/')[-1].replace('.tsv.gz', '').replace('.',"_")
    
    # add each DataFrame that was read in to the TABLES dict with key based on file name
    TABLES[key] = df

In [376]:
TABLES.keys()

dict_keys(['title_principals', 'title_akas', 'title_basics', 'title_crew', 'title_ratings', 'name_basics'])

In [377]:
## assign each DataFrame from TABLES dict to its own variable
title_principals = TABLES['title_principals']
title_akas = TABLES['title_akas']
title_basics = TABLES['title_basics']
title_crew = TABLES['title_crew']
title_ratings = TABLES['title_ratings']
name_basics = TABLES['name_basics']

## Query IMDb Data

In [378]:
check_ends(title_basics)

(7023997, 9)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
7023992,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7023993,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7023994,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7023995,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short
7023996,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014,\N,10,"Animation,Comedy,Family"


In [379]:
## only interested in movies, not tv shows, so need to be able to filter out non-movie categories
title_basics['titleType'].value_counts()

tvEpisode       5028746
short            752362
movie            556830
video            273501
tvSeries         189009
tvMovie          122764
tvMiniSeries      32547
tvSpecial         29144
videoGame         26038
tvShort           13056
Name: titleType, dtype: int64

In [380]:
## define function to query DataFrames
pysqldf = lambda q: sqldf(q, globals())

In [381]:
## select movie title IDs (tconst) from title_ratings df so that we only get movies that have ratings
## join on title_basics df to also filter movies by startYear and titleType

## only want movies made 2010 to present

q1 = """
SELECT tconst, titleType, primaryTitle, startYear, genres, averageRating 
FROM title_ratings
JOIN title_basics
USING(tconst)
WHERE startYear > 2009 
AND (titleType = 'movie' OR titleType = 'tvMovie')
"""
imdb_df = pysqldf(q1)

In [382]:
check_ends(imdb_df)

(98339, 6)

,tconst,titleType,primaryTitle,startYear,genres,averageRating
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6
1,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,2020,Drama,6.5
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7
3,tt0069049,movie,The Other Side of the Wind,2018,Drama,6.8
4,tt0071844,movie,Till Marriage Do Us Part,\N,Comedy,6.5


,tconst,titleType,primaryTitle,startYear,genres,averageRating
98334,tt9916132,movie,The Mystery of a Buryat Lama,2018,"Biography,Documentary,History",3.6
98335,tt9916160,movie,DrÃ¸mmeland,2019,Documentary,6.6
98336,tt9916192,tvMovie,Danielle Darrieux: Il est poli d'Ãªtre gai!,2019,Biography,7.4
98337,tt9916428,movie,The Secret of China,2019,"Adventure,History,War",3.5
98338,tt9916538,movie,Kuambil Lagi Hatiku,2019,Drama,8.4


### Detecting and Dealing with Missing Values

In [383]:
## no values are showing up as null values, should check for placeholder values
display(imdb_df.info())
check_df(imdb_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98339 entries, 0 to 98338
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         98339 non-null  object 
 1   titleType      98339 non-null  object 
 2   primaryTitle   98339 non-null  object 
 3   startYear      98339 non-null  object 
 4   genres         98339 non-null  object 
 5   averageRating  98339 non-null  float64
dtypes: float64(1), object(5)
memory usage: 4.5+ MB


None

,dtypes,null values,% null,nunique
tconst,object,0,0.0,98339
titleType,object,0,0.0,2
primaryTitle,object,0,0.0,92721
startYear,object,0,0.0,13
genres,object,0,0.0,974
averageRating,float64,0,0.0,91


In [384]:
## check 'startYear' for any strange values
imdb_df['startYear'].value_counts().index
# placeholder for missing values is '\N', but didn't show up as a null value

Index(['2017', '2016', '2015', '2018', '2014', '2013', '2012', '2019', '2011',
       '2010', '2020', '\N', '2021'],
      dtype='object')

The only columns with missing values are 'startYear' and 'genres'. Since 'startYear' is so relevant for informing current trends, dropping those 18 titles out of the total 98,339 is necessary. Since the questions I want to answer focus quite a lot on genre, dropping the 1,011 records missing genre information is also acceptable given that it represents such a small portion of the entire dataset.

In [385]:
# check how many missing values (placeholder '\N's) there are and where they are
for col in imdb_df.columns:
    
    ## show the head of sliced DataFrames where value in each column is '\\N'
    display(imdb_df.loc[imdb_df[col]=='\\N'].head())
    
    ## print the name of the column and the length of the dataframe created for each column
    ## (corresponds to total number of missing values for that variable)
    print('{}: {}'.format(col, len(imdb_df.loc[imdb_df[col]=='\\N'])))

,tconst,titleType,primaryTitle,startYear,genres,averageRating


tconst: 0


,tconst,titleType,primaryTitle,startYear,genres,averageRating


titleType: 0


,tconst,titleType,primaryTitle,startYear,genres,averageRating


...

genres: 1011


/Users/maxsteele/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,tconst,titleType,primaryTitle,startYear,genres,averageRating


averageRating: 0


/Users/maxsteele/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [388]:
# simpler alternative to below -- actually not working
imdb_df = imdb_df.loc[(imdb_df['startYear']!=r"\N") | (imdb_df['genres']!=r"\N")].reset_index(drop=True)
imdb_df.head()

,tconst,titleType,primaryTitle,startYear,genres,averageRating
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6
1,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,2020,Drama,6.5
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7
3,tt0069049,movie,The Other Side of the Wind,2018,Drama,6.8
4,tt0071844,movie,Till Marriage Do Us Part,\N,Comedy,6.5


In [359]:
## drop records that contain those missing values
imdb_df.drop(imdb_df.loc[imdb_df['startYear']=='\\N'].index, inplace=True)
imdb_df.drop(imdb_df.loc[imdb_df['genres']=='\\N'].index, inplace=True)

## reset index now that rows have been dropped
imdb_df = imdb_df.reset_index(drop=True)

In [360]:
# check to make sure those placeholders are gone
for col in imdb_df.columns:
    
    ## show the head of sliced DataFrames where value in each column is '\\N'
    display(imdb_df.loc[imdb_df[col]=='\\N'].head())
    
    ## print the name of the column and the length of the dataframe created for each column
    ## (corresponds to total number of missing values for that variable)
    print('{}: {}'.format(col, len(imdb_df.loc[imdb_df[col]=='\\N'])))

,tconst,titleType,primaryTitle,startYear,genres,averageRating


tconst: 0


,tconst,titleType,primaryTitle,startYear,genres,averageRating


titleType: 0


,tconst,titleType,primaryTitle,startYear,genres,averageRating


...

genres: 0


/Users/maxsteele/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,tconst,titleType,primaryTitle,startYear,genres,averageRating


averageRating: 0


/Users/maxsteele/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [361]:
## still left with 97310 movies
len(imdb_df)

97310

### Genre

In [362]:
# create 'genre_list' col where the genres are contained in a list rather than one long string
imdb_df['genres'] = imdb_df['genres']#.fillna('')
imdb_df['genre_list'] = imdb_df['genres'].apply(lambda x: x.split(','))

print(type(imdb_df['genre_list'][0]))
imdb_df['genre_list'][0]

<class 'list'>


['Comedy', 'Musical']

In [363]:
explode_genre = imdb_df.explode('genre_list') 
# could group by genre and get sum of revenue or similar aggr info
explode_genre.head(20)

,tconst,titleType,primaryTitle,startYear,genres,averageRating,genre_list
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6,Comedy
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6,Musical
1,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,2020,Drama,6.5,Drama
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7,Drama
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7,Thriller
3,tt0069049,movie,The Other Side of the Wind,2018,Drama,6.8,Drama
4,tt0100275,movie,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.6,Comedy
4,tt0100275,movie,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.6,Drama
4,tt0100275,movie,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.6,Fantasy
5,tt0112502,movie,Bigfoot,2017,"Horror,Thriller",4.5,Horror


In [364]:
explode_genre['genre_list'].value_counts()

Drama          39112
Documentary    25613
Comedy         22293
Thriller       10198
Horror          9008
Romance         8722
Action          8464
Crime           6332
Biography       4951
Adventure       4737
Family          4529
Mystery         4043
History         3827
Fantasy         2784
Sci-Fi          2643
Music           2599
Animation       2375
Sport           1535
War             1118
Musical          929
News             614
Western          349
Adult             80
Reality-TV        42
Talk-Show          2
Game-Show          1
Name: genre_list, dtype: int64

In [365]:
## need to create a column for each genre to be filled w/ boolean values based on the genre(s) of ea movie

## join all genres in the 'genres' col into one long string
all_genres_string = ','.join(imdb_df['genres'])

## split that string by commas, use set() to return only unique values, make those values into a list
all_genres_list = list(set(all_genres.split(',')))
all_genres_list = all_genres_list[1:] # first category was empty string, take it out

## use ea item in the list to make cols in df and fill that column with boolean if is/is not that genre
for genre in all_genres_list:
    imdb_df[genre] = imdb_df['genres'].str.contains(genre)

imdb_df.head()

,tconst,titleType,primaryTitle,startYear,genres,averageRating,genre_list,Crime,Comedy,Adventure,...,Game-Show,Thriller,Drama,Fantasy,Musical,Short,Biography,Animation,War,Sci-Fi
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6,"[Comedy, Musical]",False,True,False,...,False,False,False,False,True,False,False,False,False,False
1,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,2020,Drama,6.5,[Drama],False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7,"[Drama, Thriller]",False,False,False,...,False,True,True,False,False,False,False,False,False,False
3,tt0069049,movie,The Other Side of the Wind,2018,Drama,6.8,[Drama],False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,tt0100275,movie,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.6,"[Comedy, Drama, Fantasy]",False,True,False,...,False,False,True,True,False,False,False,False,False,False


In [337]:
## add a column for the number of genres a movie spans

## new 'genre_count' col is filled with the sum of all T/F values across all the genre cols for that row
## (axis=1 specifies to add across the row rather than down the col)
imdb_df['genre_count'] = imdb_df[all_genres_list].sum(axis=1)

imdb_df.head()

,tconst,titleType,primaryTitle,startYear,genres,averageRating,genre_list,Crime,Comedy,Adventure,...,Thriller,Drama,Fantasy,Musical,Short,Biography,Animation,War,Sci-Fi,genre_count
0,tt0016906,movie,Frivolinas,2014,"Comedy,Musical",5.6,"[Comedy, Musical]",False,True,False,...,False,False,False,True,False,False,False,False,False,3
1,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,2020,Drama,6.5,[Drama],False,False,False,...,False,True,False,False,False,False,False,False,False,1
2,tt0064322,movie,The Woman with the Knife,2010,"Drama,Thriller",6.7,"[Drama, Thriller]",False,False,False,...,True,True,False,False,False,False,False,False,False,2
3,tt0069049,movie,The Other Side of the Wind,2018,Drama,6.8,[Drama],False,False,False,...,False,True,False,False,False,False,False,False,False,1
4,tt0100275,movie,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.6,"[Comedy, Drama, Fantasy]",False,True,False,...,False,True,True,False,False,False,False,False,False,3


## bom.movie_gross.csv.gz

This file contains info on the year movies were made, how much they grossed within the US and how much they grossed outside the US. I have also added a 'total_gross' column that sums how much the movie grossed worldwide. Twenty-eight of the movies are missing 'domestic_gross' info. However the majority of these movies grossed very little outside of the US and thus are not likely to be very informative in terms of identifying how to make top grossing blockbuster films in and out of the US (median = \\$2.75 million, mean = $14.21 million, brought up by a  few outliers).

In [324]:
movie_gross_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz', compression='gzip')
display(movie_gross_df.shape, movie_gross_df.head(), movie_gross_df.tail())

(3387, 5)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [325]:
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [326]:
# need to convert gross values to millions for readability
movie_gross_df['domestic_gross'] = movie_gross_df['domestic_gross'] / 1000000
movie_gross_df.describe()

,domestic_gross,year
count,3359.000000,3387.000000
mean,28.745845,2013.958075
std,66.982498,2.478141
min,0.000100,2010.000000
25%,0.120000,2012.000000
50%,1.400000,2014.000000
75%,27.900000,2016.000000
max,936.700000,2018.000000


In [327]:
check_df(movie_gross_df) # foreign gross is an object which explains why it didn't show up in .describe()
# there are only 3386 unique titles out of 3387 entries, need to check for duplicates

,dtypes,null values,% null,nunique
title,object,0,0.00,3386
studio,object,5,0.15,257
domestic_gross,float64,28,0.83,1797
foreign_gross,object,1350,39.86,1204
year,int64,0,0.00,9


In [328]:
# there are no immediately identifiable duplicate entries in this DataFrame
movie_gross_df.duplicated().sum()

0

In [329]:
# which title is repeated?
movie_gross_df['title'].value_counts() # some film called Bluebeard, need to check it out

Bluebeard                                2
Post Tenebras Lux                        1
Cyrus                                    1
The Longest Ride                         1
Let's Be Cops                            1
                                        ..
The Grey                                 1
Ya Veremos                               1
The Accidental Detective 2: In Action    1
Tomorrowland                             1
No Strings Attached                      1
Name: title, Length: 3386, dtype: int64

In [330]:
# it appears these are two separate movies, one made in 2010 and the other in 2017
movie_gross_df[movie_gross_df['title']=='Bluebeard']

,title,studio,domestic_gross,foreign_gross,year
317,Bluebeard,Strand,0.0335,5200,2010
3045,Bluebeard,WGUSA,0.0431,NaN,2017


In [331]:
# rename each giving by adding the year onto the end of the title
#movie_gross_df.loc[317, 'title'] = 'Bluebeard_2010'
#movie_gross_df.loc[3045, 'title'] = 'Bluebeard_2017'

# check that it worked
#display(movie_gross_df[movie_gross_df['title']=='Bluebeard_2010'])
#movie_gross_df[movie_gross_df['title']=='Bluebeard_2017']

,title,studio,domestic_gross,foreign_gross,year
317,Bluebeard_2010,Strand,0.0335,5200,2010


,title,studio,domestic_gross,foreign_gross,year
3045,Bluebeard_2017,WGUSA,0.0431,NaN,2017


In [332]:
# there appear to be no duplicate entries in this DataFrame
movie_gross_df.duplicated().sum()

0

In [333]:
# get rid of commas in foreign_gross col and convert to floats
# but first, apparently there was a mix of floats and other dtypes in the col, so force all to be str
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].astype(str)
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].map(lambda x: x.replace(',', ''))
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].astype(float) / 1000000
# also convert foreign_gross to be in millions of dollars for readability and consistency
movie_gross_df.describe()

,domestic_gross,foreign_gross,year
count,3359.000000,2037.000000,3387.000000
mean,28.745845,74.872810,2013.958075
std,66.982498,137.410601,2.478141
min,0.000100,0.000600,2010.000000
25%,0.120000,3.700000,2012.000000
50%,1.400000,18.700000,2014.000000
75%,27.900000,74.900000,2016.000000
max,936.700000,960.500000,2018.000000


In [334]:
display(movie_gross_df.shape, movie_gross_df.head())
check_df(movie_gross_df)

(3387, 5)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415.0,652.0,2010
1,Alice in Wonderland (2010),BV,334.2,691.3,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296.0,664.3,2010
3,Inception,WB,292.6,535.7,2010
4,Shrek Forever After,P/DW,238.7,513.9,2010


,dtypes,null values,% null,nunique
title,object,0,0.00,3387
studio,object,5,0.15,257
domestic_gross,float64,28,0.83,1797
foreign_gross,float64,1350,39.86,1204
year,int64,0,0.00,9


In [335]:
# add a total_gross col to the df that adds domestic and foreign gross
# if one value is missing, treat it as 0 when adding
movie_gross_df['total_gross'] = movie_gross_df['domestic_gross'].fillna(0) \
                              + movie_gross_df['foreign_gross'].fillna(0)
display(movie_gross_df.shape, movie_gross_df.head(), movie_gross_df.tail())

(3387, 6)

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415.0,652.0,2010,1067.0
1,Alice in Wonderland (2010),BV,334.2,691.3,2010,1025.5
2,Harry Potter and the Deathly Hallows Part 1,WB,296.0,664.3,2010,960.3
3,Inception,WB,292.6,535.7,2010,828.3
4,Shrek Forever After,P/DW,238.7,513.9,2010,752.6


,title,studio,domestic_gross,foreign_gross,year,total_gross
3382,The Quake,Magn.,0.0062,NaN,2018,0.0062
3383,Edward II (2018 re-release),FM,0.0048,NaN,2018,0.0048
3384,El Pacto,Sony,0.0025,NaN,2018,0.0025
3385,The Swan,Synergetic,0.0024,NaN,2018,0.0024
3386,An Actor Prepares,Grav.,0.0017,NaN,2018,0.0017


In [336]:
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
 5   total_gross     3387 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB


In [337]:
movie_gross_df.describe()

,domestic_gross,foreign_gross,year,total_gross
count,3359.000000,2037.000000,3387.000000,3387.000000
mean,28.745845,74.872810,2013.958075,73.538001
std,66.982498,137.410601,2.478141,170.509102
min,0.000100,0.000600,2010.000000,0.000100
25%,0.120000,3.700000,2012.000000,0.274000
50%,1.400000,18.700000,2014.000000,5.475000
75%,27.900000,74.900000,2016.000000,61.350000
max,936.700000,960.500000,2018.000000,1518.900000


In [338]:
check_df(movie_gross_df)

,dtypes,null values,% null,nunique
title,object,0,0.00,3387
studio,object,5,0.15,257
domestic_gross,float64,28,0.83,1797
foreign_gross,float64,1350,39.86,1204
year,int64,0,0.00,9
total_gross,float64,0,0.00,2502


In [339]:
# check out the 28 movies that are missing domestic_gross values
## if they didn't even gross much outside the U.S., probably not going to be super informative for our purposes
# creating bool series True for NaN values  
bool_series = pd.isnull(movie_gross_df['domestic_gross'])  
    
# filtering data  
# displaying data only with Gender = NaN  
missing_dom_gross = movie_gross_df[bool_series]  
missing_dom_gross

,title,studio,domestic_gross,foreign_gross,year,total_gross
230,It's a Wonderful Afterlife,UTV,NaN,1.3000,2010,1.3000
298,Celine: Through the Eyes of the World,Sony,NaN,0.1190,2010,0.1190
302,White Lion,Scre.,NaN,0.0996,2010,0.0996
306,Badmaash Company,Yash,NaN,0.0644,2010,0.0644
327,Aashayein (Wishes),Relbig.,NaN,0.0038,2010,0.0038
537,Force,FoxS,NaN,4.8000,2011,4.8000
713,Empire of Silver,NeoC,NaN,0.0190,2011,0.0190
871,Solomon Kane,RTWC,NaN,19.6000,2012,19.6000
928,The Tall Man,Imag.,NaN,5.2000,2012,5.2000
933,Keith Lemon: The Film,NaN,NaN,4.0000,2012,4.0000


In [340]:
# most of these movies were very low grossing outside the US (median = $2.75 million, 
## mean = $14.21 million due to a handful of outliers)
### because of this, we wouldn't be losing much relevant info if these rows were ultimately dropped
### since we're presumably interested in making movies that will perform well in and out of the US
display(missing_dom_gross.describe(), movie_gross_df.describe())

,domestic_gross,foreign_gross,year,total_gross
count,0.0,28.000000,28.000000,28.000000
mean,NaN,14.214711,2012.750000,14.214711
std,NaN,31.263022,2.011541,31.263022
min,NaN,0.003800,2010.000000,0.003800
25%,NaN,0.522000,2011.750000,0.522000
50%,NaN,2.750000,2012.500000,2.750000
75%,NaN,6.250000,2014.000000,6.250000
max,NaN,122.000000,2017.000000,122.000000


,domestic_gross,foreign_gross,year,total_gross
count,3359.000000,2037.000000,3387.000000,3387.000000
mean,28.745845,74.872810,2013.958075,73.538001
std,66.982498,137.410601,2.478141,170.509102
min,0.000100,0.000600,2010.000000,0.000100
25%,0.120000,3.700000,2012.000000,0.274000
50%,1.400000,18.700000,2014.000000,5.475000
75%,27.900000,74.900000,2016.000000,61.350000
max,936.700000,960.500000,2018.000000,1518.900000


## imbd.name.basics.csv.gz

It is currently not apparent how this will fit in with the movies_gross_df above, so we'll circle back. It appears that the 'nconst' column is a unique identifier, and the 'known_for_titles' column contains movie title identifiers that match up with those used as unique identifiers for all the imdb.title DataFrames. This should allow us to connect managers/producers/actors/etc. with highly successful movies and that could help us identify some people worth reaching out to based on the types of movie(s) we set out to make.

In [341]:
name_basics_df = pd.read_csv('./zippedData/imdb.name.basics.csv.gz', compression='gzip')
display(name_basics_df.shape, name_basics_df.head(), name_basics_df.tail())

(606648, 6)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
606643,nm9990381,Susan Grobes,NaN,NaN,actress,NaN
606644,nm9990690,Joo Yeon So,NaN,NaN,actress,"tt9090932,tt8737130"
606645,nm9991320,Madeline Smith,NaN,NaN,actress,"tt8734436,tt9615610"
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer,NaN
606647,nm9993380,Pegasus Envoyé,NaN,NaN,"director,actor,writer",tt8743182


In [342]:
name_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


In [343]:
name_basics_df.describe() # not very informative

,birth_year,death_year
count,82736.000000,6783.000000
mean,1967.043826,2000.523367
std,22.122190,43.951530
min,1.000000,17.000000
25%,1957.000000,2001.000000
50%,1971.000000,2013.000000
75%,1981.000000,2016.000000
max,2014.000000,2019.000000


In [344]:
# looks like 'nconst' col serves as a unique identifier for these people that work(ed) in the movie industry
# missing most birth years and death years
# missing ~8.5% of info on primary profession
# missing ~5% of info on titles these people are known for, so might as well go ahead and drop those entries, 
## but first let's check for duplicate names
check_df(name_basics_df)

,dtypes,null values,% null,nunique
nconst,object,0,0.00,606648
primary_name,object,0,0.00,577203
birth_year,float64,523912,86.36,267
death_year,float64,599865,98.88,214
primary_profession,object,51340,8.46,8647
known_for_titles,object,30204,4.98,482207


In [345]:
# should be 0 since nconst col has only unique values
name_basics_df.duplicated().sum()

0

In [346]:
# check names that come up very frequently
name_basics_df['primary_name'].value_counts()
# the names at the top of this list that are the most common seem like very common names in general

Michael Brown       16
James Brown         16
David Brown         15
Michael Johnson     14
Mark Davis          13
                    ..
Andy Harvey          1
Jerome Pikwane       1
Stefan Manea         1
Miika Hyytiäinen     1
Malcolm Guy          1
Name: primary_name, Length: 577203, dtype: int64

In [347]:
common_names = pd.DataFrame(name_basics_df['primary_name'].value_counts())
common_names.head(20)

,primary_name
Michael Brown,16
James Brown,16
David Brown,15
Michael Johnson,14
Mark Davis,13
Dinesh,13
David Allen,12
Tim Brown,11
John Adams,11
Mark Williams,11


In [348]:
# over 21,000 rows of names that are seen more than once
# best use of time right now is to assume that nmconst truly does represent unique individuals, 
## rather than check that now
### can always double check this on a subset of people we'd actually be interested in working with
#### once we've decided what kind of movie(s) we're interested in making
common_names.loc[common_names['primary_name']>1]

,primary_name
Michael Brown,16
James Brown,16
David Brown,15
Michael Johnson,14
Mark Davis,13
...,...
Carlos González,2
Jeff Grace,2
Audrey Black,2
Richard Lester,2


In [349]:
# need to check out how the 'known_for_titles' values are entered
## since the one col contains multiple titles, we'll need some way to separate each title out to
### check each title the person worked on against variables of interest
print(type(name_basics_df['known_for_titles'][0])) # it's entered as strings
name_basics_df['known_for_titles']

<class 'str'>


0         tt0837562,tt2398241,tt0844471,tt0118553
1         tt0896534,tt6791238,tt0287072,tt1682940
2         tt1470654,tt0363631,tt0104030,tt0102898
3         tt0114371,tt2004304,tt1618448,tt1224387
4         tt0452644,tt0452692,tt3458030,tt2178256
                           ...                   
606643                                        NaN
606644                        tt9090932,tt8737130
606645                        tt8734436,tt9615610
606646                                        NaN
606647                                  tt8743182
Name: known_for_titles, Length: 606648, dtype: object

In [350]:
# it will be much easier to separate out and possibly loop through titles for each individ if necessary if
## those strings can be made into lists
# but apparently some of them were stored as floats, so have to convert all to strings first
name_basics_df['known_for_titles'] = name_basics_df['known_for_titles'].dropna().astype(str)
name_basics_df['known_for_titles'] = name_basics_df['known_for_titles'].dropna().apply(lambda x: x.split(','))
type(name_basics_df['known_for_titles'][0])

list

In [351]:
name_basics_df['known_for_titles']

0         [tt0837562, tt2398241, tt0844471, tt0118553]
1         [tt0896534, tt6791238, tt0287072, tt1682940]
2         [tt1470654, tt0363631, tt0104030, tt0102898]
3         [tt0114371, tt2004304, tt1618448, tt1224387]
4         [tt0452644, tt0452692, tt3458030, tt2178256]
                              ...                     
606643                                             NaN
606644                          [tt9090932, tt8737130]
606645                          [tt8734436, tt9615610]
606646                                             NaN
606647                                     [tt8743182]
Name: known_for_titles, Length: 606648, dtype: object

## imbd.title.akas.csv.gz

Again, it is currently not apparent how this will fit in with the movies_gross_df above, so we'll circle back. It appears to provide alternative titles (mostly titles in different languages) for the different title IDs, so title_id does not serve a a unique indentifier for this DataFrame.

In [352]:
title_akas_df = pd.read_csv('./zippedData/imdb.title.akas.csv.gz', compression='gzip')
display(title_akas_df.shape, title_akas_df.head(), title_akas_df.tail())

(331703, 8)

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


,title_id,ordering,title,region,language,types,attributes,is_original_title
331698,tt9827784,2,Sayonara kuchibiru,NaN,NaN,original,NaN,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,NaN,0.0
331700,tt9880178,1,La atención,NaN,NaN,original,NaN,1.0
331701,tt9880178,2,La atención,ES,NaN,NaN,NaN,0.0
331702,tt9880178,3,The Attention,XWW,en,imdbDisplay,NaN,0.0


In [353]:
title_akas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   title_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [354]:
title_akas_df.describe()
# not very informative
# I'm not sure what this 'ordering' column is meant to convey, but 'is_original_title' is basically True/False

,ordering,is_original_title
count,331703.000000,331678.000000
mean,5.125872,0.134769
std,6.706664,0.341477
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,6.000000,0.000000
max,61.000000,1.000000


In [355]:
display(title_akas_df.shape)
check_df(title_akas_df)

(331703, 8)

,dtypes,null values,% null,nunique
title_id,object,0,0.00,122302
ordering,int64,0,0.00,61
title,object,0,0.00,252781
region,object,53293,16.07,213
language,object,289988,87.42,76
types,object,163256,49.22,10
attributes,object,316778,95.50,77
is_original_title,float64,25,0.01,2


## imbd.title.basics.csv.gz

In [39]:
title_basics_df = pd.read_csv('./zippedData/imdb.title.basics.csv.gz', compression='gzip')
display(title_basics_df.shape, title_basics_df.head(), title_basics_df.tail())

(146144, 6)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN
146143,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,NaN,Documentary


In [357]:
title_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [358]:
title_basics_df.describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [359]:
print(title_basics_df.shape)
check_df(title_basics_df)
# missing 21.72% of runtime values which is a decent chunk
## HOWEVER, looking immediately above at ouput from .describe(), the mean runtime is extremelty close
### to the median runtime (86.2 min vs 87 minutes), so filling in null values with the median may be a
### reasonable way to avoid dropping those films from the dataset we ultimately use to address our questions
### if runtime is a variable of interest

(146144, 6)


,dtypes,null values,% null,nunique
tconst,object,0,0.00,146144
primary_title,object,0,0.00,136071
original_title,object,21,0.01,137773
start_year,int64,0,0.00,19
runtime_minutes,float64,31739,21.72,367
genres,object,5408,3.70,1085


In [363]:
title_basics_df['genres']

0           Action,Crime,Drama
1              Biography,Drama
2                        Drama
3                 Comedy,Drama
4         Comedy,Drama,Fantasy
                  ...         
146139                   Drama
146140             Documentary
146141                  Comedy
146142                     NaN
146143             Documentary
Name: genres, Length: 146144, dtype: object

In [364]:
# usefully it appears that the genres are consistently entered in alphabetical order, so each unique value
## could be it's own useful category of combined genres
### HOWEVER, it will still be useful to be able to separate out the individual genres from the combo genres,
#### so let's create a new column where the genres are contained in a list rather than one long string
print(type(title_basics_df['genres'][0]))
title_basics_df['genres'].value_counts().head(25)

<class 'str'>


Documentary                      32185
Drama                            21486
Comedy                            9177
Horror                            4372
Comedy,Drama                      3519
Thriller                          3046
Action                            2219
Biography,Documentary             2115
Drama,Romance                     2079
Comedy,Drama,Romance              1558
Documentary,Drama                 1554
Comedy,Romance                    1507
Romance                           1454
Documentary,Music                 1365
Drama,Thriller                    1335
Documentary,History               1289
Horror,Thriller                   1253
Biography,Documentary,History     1230
Biography,Documentary,Drama       1028
Family                             939
Animation                          828
Drama,Family                       784
Action,Crime,Drama                 727
Comedy,Horror                      724
Crime,Drama                        723
Name: genres, dtype: int6

In [50]:
# create 'genre_list' col where the genres are contained in a list rather than one long string
title_basics_df['genres'] = title_basics_df['genres'].fillna('')
title_basics_df['genre_list'] = title_basics_df['genres'].apply(lambda x: x.split(','))
print(type(title_basics_df['genre_list'][0]))
title_basics_df['genre_list'][0]

<class 'list'>


['Action', 'Crime', 'Drama']

In [60]:
title_basics_df.explode('genre_list') # could group by genre and get sum of revenue or similar aggr info

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre_list
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Action
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Crime
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Drama
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Drama
...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,,


In [57]:
all_genres_string = ','.join(title_basics_df['genres'])
all_genres_list = list(set(all_genres.split(',')))
all_genres_list # use these to make cols in df

['',
 'Crime',
 'Comedy',
 'Adventure',
 'News',
 'Mystery',
 'Family',
 'Action',
 'Horror',
 'Reality-TV',
 'Sport',
 'Music',
 'Documentary',
 'Adult',
 'Talk-Show',
 'Western',
 'History',
 'Romance',
 'Game-Show',
 'Thriller',
 'Drama',
 'Fantasy',
 'Musical',
 'Short',
 'Biography',
 'Animation',
 'War',
 'Sci-Fi']

In [58]:
# title_basics_df['Western'] = 
title_basics_df['genres'].str.contains(all_genres_list[-1])

0         False
1         False
2         False
3         False
4         False
          ...  
146139    False
146140    False
146141    False
146142    False
146143    False
Name: genres, Length: 146144, dtype: bool

In [366]:
title_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   genre_list       140736 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 7.8+ MB


In [367]:
# it's possible that the number of genres a movie spans could affect how well it performs
## too few genres and it doesn't offer enough for a wide audience,
### whereas maybe trying to be too many things results in a poorly put together film
#### so lets add a col based on the number of genres a movie spans

len(title_basics_df['genre_list'][0])

3

In [368]:
type(title_basics_df['genre_list'][0]) # but how do I deal with the null values???

list

In [369]:
check_df(title_basics_df.drop(columns='genre_list'))

,dtypes,null values,% null,nunique
tconst,object,0,0.00,146144
primary_title,object,0,0.00,136071
original_title,object,21,0.01,137773
start_year,int64,0,0.00,19
runtime_minutes,float64,31739,21.72,367
genres,object,5408,3.70,1085


## imbd.title.crew.csv.gz

This DataFrame associates the movie titles, based on their unique identifiers for title_id/tconst, with their directors and writers, based on the unique identifiers for people in the industry (nmconst).

In [370]:
title_crew_df = pd.read_csv('./zippedData/imdb.title.crew.csv.gz', compression='gzip')
display(title_crew_df.shape, title_crew_df.head(), title_crew_df.tail())

(146144, 3)

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


,tconst,directors,writers
146139,tt8999974,nm10122357,nm10122357
146140,tt9001390,nm6711477,nm6711477
146141,tt9001494,"nm10123242,nm10123248",NaN
146142,tt9004986,nm4993825,nm4993825
146143,tt9010172,NaN,nm8352242


In [371]:
title_crew_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [372]:
check_df(title_crew_df)

,dtypes,null values,% null,nunique
tconst,object,0,0.00,146144
directors,object,5727,3.92,98525
writers,object,35883,24.55,91920


## imbd.title.principals.csv.gz

It appears that this DataFrame list some of the majot people who worked on/ performed in certain movies (directors, writers, producers, actors, etc.) That is the information provided by the category column. The 'tconst' column is again the unique identifier for movies, the 'nmconst' column is the same unique identifier for people in the industry. The 'job' and 'characters' columns are missing a majority of the values.

In [373]:
title_principals_df = pd.read_csv('./zippedData/imdb.title.principals.csv.gz', compression='gzip')
display(title_principals_df.shape, title_principals_df.head(), title_principals_df.tail())

(1028186, 6)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


,tconst,ordering,nconst,category,job,characters
1028181,tt9692684,1,nm0186469,actor,NaN,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,NaN,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,NaN,NaN
1028184,tt9692684,4,nm6009913,writer,writer,NaN
1028185,tt9692684,5,nm10441595,producer,producer,NaN


In [374]:
title_principals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   tconst      1028186 non-null  object
 1   ordering    1028186 non-null  int64 
 2   nconst      1028186 non-null  object
 3   category    1028186 non-null  object
 4   job         177684 non-null   object
 5   characters  393360 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


In [375]:
check_df(title_principals_df)

,dtypes,null values,% null,nunique
tconst,object,0,0.00,143454
ordering,int64,0,0.00,10
nconst,object,0,0.00,604546
category,object,0,0.00,12
job,object,850502,82.72,2965
characters,object,634826,61.74,174762


## imbd.title.ratings.csv.gz

In [376]:
title_ratings_df = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz', compression='gzip')
display(title_ratings_df.shape, title_ratings_df.head(), title_ratings_df.tail())

(73856, 3)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


,tconst,averagerating,numvotes
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5
73855,tt9894098,6.3,128


In [377]:
title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [378]:
title_ratings_df.describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [379]:
check_df(title_ratings_df)

,dtypes,null values,% null,nunique
tconst,object,0,0.0,73856
averagerating,float64,0,0.0,91
numvotes,int64,0,0.0,7349


In [381]:
title_ratings_df['averagerating'].value_counts()

7.0     2262
6.6     2251
7.2     2249
6.8     2239
6.5     2221
        ... 
9.6       18
10.0      16
9.8       15
9.7       12
9.9        5
Name: averagerating, Length: 91, dtype: int64

In [382]:
title_ratings_df['numvotes'].value_counts()

6        2875
5        2699
7        2476
8        2167
9        1929
         ... 
18286       1
16289       1
1958        1
4007        1
4098        1
Name: numvotes, Length: 7349, dtype: int64

## rt.movie_info.tsv.gz

In [383]:
rt_movie_df = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t', compression='gzip')
display(rt_movie_df.shape, rt_movie_df.head(), rt_movie_df.tail())

(1560, 12)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN
1559,2000,"Suspended from the force, Paris cop Hubert is ...",R,Action and Adventure|Art House and Internation...,NaN,Luc Besson,"Sep 27, 2001","Feb 11, 2003",NaN,NaN,94 minutes,Columbia Pictures


In [384]:
rt_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [386]:
check_df(rt_movie_df)

,dtypes,null values,% null,nunique
id,int64,0,0.00,1560
synopsis,object,62,3.97,1497
rating,object,3,0.19,6
genre,object,8,0.51,299
director,object,199,12.76,1125
writer,object,449,28.78,1069
theater_date,object,359,23.01,1025
dvd_date,object,359,23.01,717
currency,object,1220,78.21,1
box_office,object,1220,78.21,336


## rt.reviews.tsv.gz

In [418]:
rt_reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', compression='gzip', \
                            encoding = "ISO-8859-1", engine='python') 
# ran into issues trying to decode with encoding="utf-8"
display(rt_reviews_df.shape, rt_reviews_df.head(), rt_reviews_df.tail())

(54432, 8)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


,id,review,rating,fresh,critic,top_critic,publisher,date
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"
54431,2000,NaN,3/5,fresh,Nicolas Lacroix,0,Showbizz.net,"November 12, 2002"


In [419]:
rt_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [423]:
print(rt_reviews_df.shape)
check_df(rt_reviews_df)

(54432, 8)


,dtypes,null values,% null,nunique
id,int64,0,0.00,1135
review,object,5563,10.22,48682
rating,object,13517,24.83,186
fresh,object,0,0.00,2
critic,object,2722,5.00,3496
top_critic,int64,0,0.00,2
publisher,object,309,0.57,1281
date,object,0,0.00,5963


## tmbd.movies.csv.gz

In [392]:
tmdb_movies_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz', compression='gzip')
display(tmdb_movies_df.shape, tmdb_movies_df.head(), tmdb_movies_df.tail())

(26517, 10)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.6,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.6,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.6,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.6,2018-06-22,Trailer Made,0.0,1
26516,26516,"[53, 27]",309885,en,The Church,0.6,2018-10-05,The Church,0.0,1


## tn.movie_budgets.csv.gz

In [424]:
movie_budgets_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz', compression='gzip')
display(movie_budgets_df.shape, movie_budgets_df.head(), movie_budgets_df.tail())

(5782, 6)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


In [425]:
movie_budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [426]:
movie_budgets_df.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [427]:
check_df(movie_budgets_df)

,dtypes,null values,% null,nunique
id,int64,0,0.0,100
release_date,object,0,0.0,2418
movie,object,0,0.0,5698
production_budget,object,0,0.0,509
domestic_gross,object,0,0.0,5164
worldwide_gross,object,0,0.0,5356


In [429]:
movie_budgets_df.duplicated().sum()

0

In [433]:
movie_budgets_df['movie'].value_counts().head(40)

King Kong                        3
Halloween                        3
Home                             3
Crash                            2
Total Recall                     2
Unforgettable                    2
The Jungle Book                  2
Trance                           2
Death at a Funeral               2
The Texas Chainsaw Massacre      2
Alice in Wonderland              2
Brothers                         2
Robin Hood                       2
The Mummy                        2
The Avengers                     2
Hellboy                          2
The Island                       2
The Fog                          2
The Alamo                        2
The Gambler                      2
Ca$h                             2
Around the World in 80 Days      2
Hercules                         2
Peteâs Dragon                  2
The Karate Kid                   2
Point Break                      2
The Signal                       2
The Day the Earth Stood Still    2
Heist               

In [434]:
movie_budgets_df = movie_budgets_df.drop(columns='id')
check_df(movie_budgets_df)

,dtypes,null values,% null,nunique
release_date,object,0,0.0,2418
movie,object,0,0.0,5698
production_budget,object,0,0.0,509
domestic_gross,object,0,0.0,5164
worldwide_gross,object,0,0.0,5356


In [435]:
movie_budgets_df.duplicated().sum()

0

In [437]:
movie_budgets_df.loc[movie_budgets_df['movie']=='King Kong']

,release_date,movie,production_budget,domestic_gross,worldwide_gross
39,"Dec 14, 2005",King Kong,"$207,000,000","$218,080,025","$550,517,357"
2374,"Dec 17, 1976",King Kong,"$23,000,000","$52,614,445","$90,614,445"
5396,"Apr 7, 1933",King Kong,"$672,000","$10,000,000","$10,000,650"


In [438]:
movie_budgets_df.loc[movie_budgets_df['movie']=='Halloween']

,release_date,movie,production_budget,domestic_gross,worldwide_gross
3014,"Aug 31, 2007",Halloween,"$15,000,000","$58,269,151","$77,514,401"
3535,"Oct 19, 2018",Halloween,"$10,000,000","$159,342,015","$254,900,667"
5536,"Oct 17, 1978",Halloween,"$325,000","$47,000,000","$70,000,000"


In [406]:
help(csv_to_sqlite)

Help on module csv_to_sqlite:

NAME
    csv_to_sqlite - A script that processes the input CSV files and copies them into a SQLite database.

CLASSES
    builtins.object
        CsvFileInfo
        CsvOptions
    
    class CsvFileInfo(builtins.object)
     |  Methods defined here:
     |  
     |  __enter__(self)
     |  
     |  __exit__(self, *args)
     |  
     |  __init__(self, path, options=None)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  determine_types(self)
     |  
     |  get_minimal_type(self, value)
     |  
     |  get_restarted_reader(self)
     |  
     |  get_table_name(self)
     |  
     |  save_to_db(self, connection)
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
 

In [ ]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
input_files = ["abilities.csv", "moves.csv"] # pass in a list of CSV files
csv_to_sqlite.write_csv(input_files, "output.sqlite", options)